# Workflow to find F1 score between manual annotation evaluation of IHC image and segmentation result of inferred H&E image from CODA segmentation model:

### Part 0: Code to save the test images that were chosen for manual annotation evaluation in a separate folder, the segmentation map of these inferred images will also be in those respective folders:

In [2]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import shutil
import matplotlib.pyplot as plt

In [14]:
src_path = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg"
files_src = [os.path.join(src_path,x) for x in os.listdir(src_path)]
files_src = [x for x in files_src if x.split("_")[-1][:3] == "IHC"] # all six folders
dst_path = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\test\manual_annotation_eval"
img_src_path = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\test"
for folder in tqdm(files_src):
    dst_folder = os.path.join(dst_path,os.path.basename(folder))
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    img_list = [os.path.join(folder,x) for x in os.listdir(folder) if x.endswith(".jpg")]
    for img in tqdm(img_list):
        shutil.copy(img,dst_folder)


100%|██████████| 6/6 [00:03<00:00,  1.77it/s]


### For conditional I2SB inference only, need HE_segmentation map so move the corresponding seg maps as well:

In [3]:
seg_map_dir = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\test\HE\classification_12082023\color"
src = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg"
file_src = [os.path.join(src,x) for x in os.listdir(src)]
file_src = [x for x in file_src if x.split("_")[-1][:3] == "IHC"] # all six folders
save_dir = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\test\manual_annotation_eval"

In [4]:
for file in tqdm(file_src):
    folder_name = os.path.basename(file)
    save_dir_folder = os.path.join(save_dir,folder_name)
    save_dir_folder = os.path.join(save_dir_folder,"HE_segmentation_map")
    if not os.path.exists(save_dir_folder):
        os.makedirs(save_dir_folder)
    all_image_list = [os.path.join(file,x) for x in os.listdir(file) if x.endswith(".jpg")]
    for images in tqdm(all_image_list):
        image_name = os.path.basename(images)
        if "IHCA" in file:
            image_name = image_name.replace("ISLET","HE")
            image_name = image_name.replace(".jpg",".png")
        elif "IHCB" in file:
            image_name = image_name.replace("VESSEL","HE")
            image_name = image_name.replace(".jpg",".png")
        elif "IHCC" in file:
            image_name = image_name.replace("IMMUNE","HE")
            image_name = image_name.replace(".jpg",".png")
        seg_map_img = os.path.join(seg_map_dir,image_name)
        shutil.copy(seg_map_img,save_dir_folder)

100%|██████████| 6/6 [00:05<00:00,  1.04it/s]


### Part 1: Code to turn manual IHC annotations saved in .xml file to a binary mask and a tissue map (Need to run only once):

In [5]:
src = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg"
file_src = [os.path.join(src,x) for x in os.listdir(src)]
file_src = [x for x in file_src if x.split("_")[-1][:3] == "IHC"] # all six folders

In [15]:
# First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data=coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return (coord_df)

In [16]:
# Then, input xml_path to use xml_to_df function to output X,Y coordinates for each annotation per class:
def coord_to_multiclass_df(xml_path):
    coord_df = xml_to_df(xml_path)
    coord_df = coord_df.drop(columns=["ID"])
    dict = {"islet": 1, "duct": 2, "vessels": 3, "fat": 4, "acini": 5, "ecm": 6, "whitespace": 7,
            "nerves": 8}
    coord_df = coord_df.replace({"ClassNames": dict})
    return coord_df

In [31]:
# Then input original image and the coord_df to output the mask with unique annotations (1..N, N = 8 in this case):
def create_mask_multi_annot(xml_path, image_size = (256,256)):
    mask = np.zeros(image_size, dtype=np.uint8)
    iter_order = [6, 5, 4, 1, 2, 3, 8, 7] #[ecm acini fat islet duct vessels nerves whitespace/noise]
    coord_df = coord_to_multiclass_df(xml_path)  #use function above
    for i in iter_order:
        coord_df_tmp = coord_df[coord_df.ClassNames == i]
        for idx, row in coord_df_tmp.iterrows():
            xx = row.X.astype(float).astype('int32')
            yy = row.Y.astype(float).astype('int32')
            contours = np.array(list(zip(xx, yy)))
            class_number = row.ClassNames
            mask = cv2.fillPoly(mask, pts=[contours.astype(int)], color=(int(class_number)))
            binary_mask = mask > 0
            binary_mask = binary_mask.astype(np.uint8)
    return mask, binary_mask

In [59]:
# for loop of using create_binary_mask_label to save binary and image_mask in the annotation folder:
for file in tqdm(file_src):
    xml_path = [os.path.join(file,x) for x in os.listdir(file) if x.endswith(".xml")]
    img_path = [os.path.join(file,x) for x in os.listdir(file) if x.endswith(".jpg")]
    if len(xml_path) != len(img_path):
        assert("xml and .jpg files mismatch")
    mask_save_path = os.path.join(file,"mask")
    bin_mask_save_path = os.path.join(file,"bin_mask")
    if not os.path.exists(mask_save_path):
        os.makedirs(mask_save_path)
    if not os.path.exists(bin_mask_save_path):
        os.makedirs(bin_mask_save_path)
    for xml_file in tqdm(xml_path):
        image_name = os.path.basename(xml_file).replace("xml","png")
        mask_test, bin_mask_test = create_mask_multi_annot(xml_file)
        Image.fromarray(mask_test).save(os.path.join(mask_save_path,image_name))
        Image.fromarray(bin_mask_test).save(os.path.join(bin_mask_save_path,image_name))

100%|██████████| 6/6 [00:05<00:00,  1.07it/s]


### Part 2: Code to use the above saved binary mask and apply it to the pix2pix, pyramid-pix2pix, and I2SB and I2SB-cond segmentation map:
### Note: First need to run segmentation on MATLAB to get the tissue map of the inferred pix2pix/i2sb:

### For pix2pix:

In [44]:
def rename_images(image_path):
    file_list = [os.path.join(image_path, x) for x in os.listdir(image_path) if x.endswith(".png")]
    for filename in tqdm(file_list):
        if "IMMUNE" in filename:
            os.rename(filename, filename.replace("IMMUNE","HE"))

In [45]:
rename_images(r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg\inferred\pix2pix\classification_01032024")

100%|██████████| 120/120 [00:00<00:00, 201.98it/s]


In [46]:
# for the file name, for IHCA, replace islet with HE, for B, replace vessel with HE, for C, replace immune with HE for file base name:
p2p_mask_dir = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg\inferred\pix2pix\classification_01032024" # path directory to tissue map of the pix2pix generated images
for file in tqdm(file_src):
    bin_mask_file = os.path.join(file,"bin_mask")
    bin_mask_path_src = [os.path.join(bin_mask_file,x) for x in os.listdir(bin_mask_file) if x.endswith(".png")]
    infer_mask_save_path = os.path.join(file,"pix2pix_mask")
    if not os.path.exists(infer_mask_save_path):
        os.makedirs(infer_mask_save_path)
    for bin_mask_path in tqdm(bin_mask_path_src):
        mask_name = os.path.basename(bin_mask_path)
        if "IHCA" in file:
            file_name = mask_name.replace("ISLET","HE")
            # file_name = file_name.replace(".png","_output.png")
        elif "IHCB" in file:
            file_name = mask_name.replace("VESSEL","HE")
            # file_name = file_name.replace(".png","_output.png")
        elif "IHCC" in file:
            file_name = mask_name.replace("IMMUNE","HE")
            # file_name = file_name.replace(".png","_output.png")
        infer_mask_path = os.path.join(p2p_mask_dir,file_name)
        infer_mask = np.array(cv2.imread(infer_mask_path,0))
        bin_mask = np.array(cv2.imread(bin_mask_path,0))
        infer_mask_edit = infer_mask * bin_mask
        Image.fromarray(infer_mask_edit).save(os.path.join(infer_mask_save_path,file_name))

100%|██████████| 6/6 [00:04<00:00,  1.40it/s]


### For pyramid-pix2pix:

### Since it was trained on 1024 x 1024, need to first get the 256x256 portion of the image and then its respective mask saved elsewhere to do the same for p-pix2pix. Below is the code to extract the respective mask:

In [66]:
# for the file name, for IHCA, replace islet with HE, for B, replace vessel with HE, for C, replace immune with HE for file base name:
p_p2p_mask_dir = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg\p-pix2pix-inferred\classification_01042024"# path directory to tissue map of the pyramid-pix2pix generated images (note these are 1024 x 1024 unlike all others)
for file in tqdm(file_src):
    bin_mask_file = os.path.join(file,"bin_mask")
    bin_mask_path_src = [os.path.join(bin_mask_file,x) for x in os.listdir(bin_mask_file) if x.endswith(".png")]
    infer_mask_save_path = os.path.join(file,"pyramid_pix2pix_mask")
    if not os.path.exists(infer_mask_save_path):
        os.makedirs(infer_mask_save_path)
    for bin_mask_path in tqdm(bin_mask_path_src):
        mask_name = os.path.basename(bin_mask_path)
        if "IHCA" in file:
            file_name = mask_name.replace("ISLET_","")
        elif "IHCB" in file:
            file_name = mask_name.replace("VESSEL_","")
        elif "IHCC" in file:
            file_name = mask_name.replace("IMMUNE_","")
        infer_mask_path = os.path.join(p_p2p_mask_dir,file_name)
        real_infer_mask_path = infer_mask_path[:-7]+".png"
        x = int(infer_mask_path[-6:-5])
        y = int(infer_mask_path[-5:-4])
        infer_mask = np.array(cv2.imread(real_infer_mask_path,0))
        infer_mask = infer_mask[(y-1)*256:y*256,(x-1)*256:x*256]
        bin_mask = np.array(cv2.imread(bin_mask_path,0))
        infer_mask_edit = infer_mask * bin_mask
        Image.fromarray(infer_mask_edit).save(os.path.join(infer_mask_save_path,file_name))

100%|██████████| 6/6 [00:03<00:00,  1.58it/s]


### For I2SB:


In [6]:
file_src

['\\\\10.99.68.178\\ashleyex\\Type_1_diabetes\\IHC to HE model\\annotation_jpg\\S1_IHCA',
 '\\\\10.99.68.178\\ashleyex\\Type_1_diabetes\\IHC to HE model\\annotation_jpg\\S1_IHCB',
 '\\\\10.99.68.178\\ashleyex\\Type_1_diabetes\\IHC to HE model\\annotation_jpg\\S1_IHCC',
 '\\\\10.99.68.178\\ashleyex\\Type_1_diabetes\\IHC to HE model\\annotation_jpg\\S2_IHCA',
 '\\\\10.99.68.178\\ashleyex\\Type_1_diabetes\\IHC to HE model\\annotation_jpg\\S2_IHCB',
 '\\\\10.99.68.178\\ashleyex\\Type_1_diabetes\\IHC to HE model\\annotation_jpg\\S2_IHCC']

In [7]:
# for the file name, for IHCA, replace islet with HE, for B, replace vessel with HE, for C, replace immune with HE for file base name:
i2sb_mask_dir = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg\inferred\I2SB\test-run-4\iter_24388\classification_01022024"
for file in tqdm(file_src):
    bin_mask_file = os.path.join(file,"bin_mask")
    bin_mask_path_src = [os.path.join(bin_mask_file,x) for x in os.listdir(bin_mask_file) if x.endswith(".png")]
    infer_mask_save_path = os.path.join(file,"i2sb_infer_seg_map")
    if not os.path.exists(infer_mask_save_path):
        os.makedirs(infer_mask_save_path)
    for bin_mask_path in tqdm(bin_mask_path_src):
        mask_name = os.path.basename(bin_mask_path)
        if "IHCA" in file:
            file_name = mask_name.replace("ISLET","HE")
            file_name = file_name.replace(".png","_inferred.png")
        elif "IHCB" in file:
            file_name = mask_name.replace("VESSEL","HE")
            file_name = file_name.replace(".png","_inferred.png")
        elif "IHCC" in file:
            file_name = mask_name.replace("IMMUNE","HE")
            file_name = file_name.replace(".png","_inferred.png")
        infer_mask_path = os.path.join(i2sb_mask_dir,file_name)
        infer_mask = np.array(cv2.imread(infer_mask_path,0))
        bin_mask = np.array(cv2.imread(bin_mask_path,0))
        infer_mask_edit = infer_mask * bin_mask
        Image.fromarray(infer_mask_edit).save(os.path.join(infer_mask_save_path,file_name))

100%|██████████| 6/6 [00:05<00:00,  1.11it/s]


### For I2SB-cond:


In [8]:
# for the file name, for IHCA, replace islet with HE, for B, replace vessel with HE, for C, replace immune with HE for file base name:
i2sb_mask_dir = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\annotation_jpg\inferred\I2SB-cond\test-run-seg-map-cond\iter_19656\classification_01022024"
for file in tqdm(file_src):
    bin_mask_file = os.path.join(file,"bin_mask")
    bin_mask_path_src = [os.path.join(bin_mask_file,x) for x in os.listdir(bin_mask_file) if x.endswith(".png")]
    infer_mask_save_path = os.path.join(file,"i2sb_cond_infer_seg_map")
    if not os.path.exists(infer_mask_save_path):
        os.makedirs(infer_mask_save_path)
    for bin_mask_path in tqdm(bin_mask_path_src):
        mask_name = os.path.basename(bin_mask_path)
        if "IHCA" in file:
            file_name = mask_name.replace("ISLET","HE")
            file_name = file_name.replace(".png","_inferred.png")
        elif "IHCB" in file:
            file_name = mask_name.replace("VESSEL","HE")
            file_name = file_name.replace(".png","_inferred.png")
        elif "IHCC" in file:
            file_name = mask_name.replace("IMMUNE","HE")
            file_name = file_name.replace(".png","_inferred.png")
        infer_mask_path = os.path.join(i2sb_mask_dir,file_name)
        infer_mask = np.array(cv2.imread(infer_mask_path,0))
        bin_mask = np.array(cv2.imread(bin_mask_path,0))
        infer_mask_edit = infer_mask * bin_mask
        Image.fromarray(infer_mask_edit).save(os.path.join(infer_mask_save_path,file_name))

100%|██████████| 6/6 [00:03<00:00,  1.53it/s]
